In [484]:
!pip install pyinterval

In [485]:
from interval import interval, imath
import pandas as pd
import math

In [486]:
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_middle_point(interv):
    return (interv[0].sup + interv[0].inf) / 2

def to_string(interv):
    return f"[{interv[0].inf:4.3f}, {interv[0].sup:4.3f}]"

In [487]:
def calculate_volterra1(f, expected, k, y0, a, b, c, d, p):
    h = (b - a) / p
    x = []
    y = []
    exp = []
    exp.append(interval[expected(a),expected(a)])
    for i in range(0, p):
        xi = a + i*h
        xinext = xi + h
        x.append(interval[xi, xinext])
        sum = 0
        for j in range(0, i):
            sum += k(x[i], x[j], y0[j])
        y.append(f(x[i]) + h*sum + interval[0,h]*k(x[i], x[i], y0[i]))
        exp.append(expected(xi))
    u = []
    u.append(y0[0])
    for i in range(0,p-1):
        u.append(y[i]&y[i+1])
    return x, y, u, exp

In [492]:
a = 0
b = 0.9
c = 0
d = 10
p = 3
n = 5
y0 = []
for i in range(p):
    y0.append(interval[c,d])
func = lambda x: x*x
k = lambda x,s,y : (x-s)*y
expected = lambda x: -2+math.exp(-x)+math.exp(x)

In [493]:
for iteration in range(n):
    print(f"\nn={iteration+1}")
    x, y, u, exp = calculate_volterra1(func, expected, k, y0, a, b, c, d, p)
    tbl = pd.DataFrame(columns=['xi', 'yi', 'ui', 'm(ui)', 'expected', 'e'])
    for i in range(p):
        mid_point=calculate_middle_point(u[i])
        real = expected(x[i][0].inf)
        tbl.loc[len(tbl)] = [to_string(x[i]),to_string(y[i]),to_string(u[i]),f"{mid_point:4.4}",f"{real:4.4}",f"{abs(mid_point-real):4.4}"]
    y0 = y
    print(tbl)


n=1
               xi               yi               ui m(ui) expected         e
0  [0.000, 0.300]  [-0.900, 0.990]  [0.000, 10.000]   5.0      0.0       5.0
1  [0.300, 0.600]  [-0.810, 3.060]  [-0.810, 0.990]  0.09  0.09068  0.000677
2  [0.600, 0.900]  [-0.540, 6.210]  [-0.540, 3.060]  1.26   0.3709    0.8891

n=2
               xi               yi               ui   m(ui) expected        e
0  [0.000, 0.300]  [-0.089, 0.179]  [-0.900, 0.990]   0.045      0.0    0.045
1  [0.300, 0.600]  [-0.347, 0.814]  [-0.089, 0.179]   0.045  0.09068  0.04568
2  [0.600, 0.900]  [-0.588, 2.187]  [-0.347, 0.814]  0.2331   0.3709   0.1378

n=3
               xi               yi               ui    m(ui) expected        e
0  [0.000, 0.300]  [-0.016, 0.106]  [-0.089, 0.179]    0.045      0.0    0.045
1  [0.300, 0.600]   [0.001, 0.465]   [0.001, 0.106]  0.05343  0.09068  0.03725
2  [0.600, 0.900]   [0.077, 1.202]   [0.077, 0.465]    0.271   0.3709  0.09991

n=4
               xi               yi          